## <center>Poshbot</center>

In [1]:
# intents
# 1. Greetings
# 2. enrollment
# 3. pricing
# 4. Track
# 5. Fallback_input

In [1]:
# Intent mapping
target_response = {
    0: 'greetings',
    1: 'enrollment',
    2: 'pricing',
    3: 'tracking',
    4: 'fallback'
}

# Response templates
fallback_responses = [
    "Sorry, I didn’t quite understand that. Can you rephrase, read my previous prompt and respond accordingly?",
    "Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.",
    "Let’s try that again. Ask about courses, prices, or tracking progress, read my previous prompt and respond accordingly"
]

response = {
    'greetings': [
        "Nice, you are very much welcome to Poshem! What do you want to do?\n"
        "- Register for a course?\n- Check the price of our courses?\n- Confirm your registered courses?"
    ],
    'enrollment': (
        "Kindly request for the course you want to register in this format:\n"
        "(e.g.) I am Promise, Date of birth, B-Tech Computer Science.\n"
        "phone number: 07063083925.\n"
        "I want to learn Excel."
    ),
    'pricing': "Our pricing varies by course. Here is the breakdown...",
    'tracking': "To track your course, kindly let me know your ID.",
    'fallback': fallback_responses
}

In [119]:
#---------------------------------------------------------USER PROFILE EXTRACTOR-----------------------------------------

# Load spaCy model
nlp = spacy.load('en_core_web_lg') 
# Remove default NER to use custom recognition
if "ner" in nlp.pipe_names:
    nlp.remove_pipe("ner")

# Add EntityRuler for course and email
ruler = nlp.add_pipe("entity_ruler", first=True)

# Define custom course list
courses = [
    "Excel", "Powerbi", "SQL", "PostgresSql", "Python Beginners",
    "Advance Python for Data Analysis", "Python for Data Analysis",
    "Data Science", "Machine Learning",
    "Database Administrator", "Azure DevOps", "AWS Devops"
]

course_patterns = [{"label": "COURSE", "pattern": course} for course in courses]
email_patterns = [{"label": "EMAIL", "pattern": [{"LIKE_EMAIL": True}]}]
ruler.add_patterns(course_patterns + email_patterns)

# Add Matcher for name, DOB, qualification, and phone
matcher = Matcher(nlp.vocab)

matcher.add("NAME", [[
    {"LOWER": "i"},
    {"LOWER": "a"},
    {"IS_TITLE": True, "OP": "+"}
]])

matcher.add("DOB", [[
    {"LOWER": "born"},
    {"LOWER": "on"},
    {"IS_TITLE": True, "OP": "+"},
    {"TEXT": {"REGEX": r"^\d{1,2}(st|nd|rd|th)?$"}, "OP": "?"},
    {"IS_DIGIT": True, "OP": "?"}
]])

matcher.add("PHONE", [[
    {"LOWER": "phone"},
    {"LOWER": "number"},
    {"IS_PUNCT": True, "OP": "?"},
    {"TEXT": {"REGEX": r"^\d{11}$"}}
]])

matcher.add("QUALIFICATION", [[
    {"TEXT": {"REGEX": r"^[A-Z][-.A-Za-z]+$"}},
    {"IS_TITLE": True, "OP": "+"}
]])

#-------------------------------------------------------------------BOT FUNCTIONS-----------------------------------------

# Function to simulate intent prediction
def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    predicted_intent = loaded_pipeline.predict([vector])[0]  # Extract single prediction
    return predicted_intent

# Function to generate bot response
def response_function(user_intent):
    intent_label = target_response.get(user_intent, 'fallback')

    if intent_label == 'fallback':
        return np.random.choice(response['fallback'])

    reply = response.get(intent_label)
    return reply[0] if isinstance(reply, list) else reply

# Chat history logging function
def log_interaction(user_input, predicted_intent, bot_response):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_intent": target_response.get(predicted_intent, 'fallback'),
        "bot_response": bot_response
    }
    logging.info(json.dumps(log_entry))

#-----------------------------------------------------------------USER EXTRACTION FUNCTION-------------------------------------

# Function to extract profile data from user input
def extract_profile(user_input):
    doc = nlp(user_input)

    # Apply matcher
    matches = matcher(doc)

    # Initialize extracted data
    name, dob, qualification, phone_number, email, course = None, None, None, None, None, []

    # Process matches
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]

        if label == "NAME":
            name = span.text.replace("I am", "").strip()
        elif label == "DOB":
            dob = span.text.replace("born on", "").strip()
        elif label == "QUALIFICATION":
            qualification = span.text
        elif label == "PHONE":
            phone_number = span[-1].text

    # Extract email & course via ruler
    for ent in doc.ents:
        if ent.label_ == "EMAIL":
            email = ent.text
        elif ent.label_ == "COURSE":
            course.append(ent.text)

    return name, dob, qualification, phone_number, email, course

#--------------------------------------------------------POSHBOT------------------------------------------------
    

def Poshbot():
    print("Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:")

    # Variable to track the state of registration
    in_registration = False
    completed_registration = False
    while True:
        if completed_registration == False:
            user_input = input("You 🫥: ").strip()
    
        if user_input.lower() in ["i am done", "exit", "quit"]:
            print("Bot 🤖: Thanks for choosing Poshem Technologies Institution. This is your course ID. You'll get a mail from us shortly")
            break
    
        try:
            predicted_intent = user_input_to_model(user_input)
            bot_response = response_function(predicted_intent)
            
            print(f"Bot 🤖: {bot_response} {in_registration}") ### print the intent response

            # If user is in the registration phase, extract profile details
            if predicted_intent == 'enrollment':
                in_registration = True
                try:
                    name, dob, qualification, phone_number, email, course = extract_profile(user_input)
                    # Save the extracted details into appropriate variables
                    print(f"Bot 🤖: Thank you for the details! Here's your profile: ")
                    print(f"Name: {name}")
                    print(f"DOB: {dob}")
                    print(f"Qualification: {qualification}")
                    print(f"Phone: {phone_number}")
                    print(f"Email: {email}")
                    print(f"Course: {course}")
                    # Complete registration or proceed with the next steps
                    
                    print("Bot 🤖: Your registration is complete!")
                except Exception as e:
                    print("Bot 🤖: Sorry, I couldn't process your enrollment details. Please try again.")

            if in_registration == True:
                print("Bot 🤖: What else do you want to do, else kindly say i am doneexit or quit")
                
            # Log the conversation
            log_interaction(user_input, predicted_intent, bot_response)

        except Exception as e:
            print("Bot 🤖: Sorry, I couldn't process that. Please try again or ask for help.")
            

In [121]:
def Poshbot():
    print("Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:")

    in_registration = False
    completed_registration = False
    predicted_intent = None

    while True:
        user_input = input("You 🫥: ").strip()

        if user_input.lower() in ["i am done", "exit", "quit"]:
            print("Bot 🤖: Thanks for choosing Poshem Technologies Institution. This is your course ID. You'll get a mail from us shortly.")
            break

        try:
            # Step 1: If user is in registration mode, extract profile
            if in_registration and not completed_registration:
                try:
                    name, dob, qualification, phone_number, email, course = extract_profile(user_input)

                    print("Bot 🤖: Thank you! Here's your profile:")
                    print(f"Name: {name}")
                    print(f"DOB: {dob}")
                    print(f"Qualification: {qualification}")
                    print(f"Phone: {phone_number}")
                    print(f"Email: {email}")
                    print(f"Course: {course}")

                    print("Bot 🤖: Your registration is complete! ✅")

                    print("Bot 🤖: What else do you want to do, else kindly quite, exit or say you are done!!!")

                    completed_registration = True
                    in_registration = False  # Reset flag
                    predicted_intent = None  # Clear intent

                    continue  # Go back to next user input
                except Exception as e:
                    print("Bot 🤖: Sorry, I couldn't extract your enrollment details. Please try again.")
                    continue

            # Step 2: Predict intent if not in registration
            predicted_intent = user_input_to_model(user_input)
            bot_response = response_function(predicted_intent)

            print(f"Bot 🤖: {bot_response}")

            # Step 3: If intent is enrollment, set the flag
            if predicted_intent == 1:  # Or 'enrollment' depending on how your model works
                in_registration = True
                continue

            # Step 4: Handle other general replies
            log_interaction(user_input, predicted_intent, bot_response)

        except Exception as e:
            print("Bot 🤖: Sorry, I couldn't process that. Please try again.")
Poshbot()

Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:


You 🫥:  hi


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Register for a course?
- Check the price of our courses?
- Confirm your registered courses?


You 🫥:  registration


Bot 🤖: Kindly request for the course you want to register in this format:
(e.g.) I am Promise, Date of birth, B-Tech Computer Science.
phone number: 07063083925.
I want to learn Excel.


You 🫥:  I am Promise, January 8th 1995, B-Tech Computer Science. phone number: 07063083925. I want to learn Excel. promiseibediogwu@gmail.com


Bot 🤖: Thank you! Here's your profile:
Name: Promise
DOB: None
Qualification: Computer Science
Phone: 07063083925
Email: promiseibediogwu@gmail.com
Course: ['Excel']
Bot 🤖: Your registration is complete! ✅
Bot 🤖: What else do you want to do, else kindly quite, exit or say you are done!!!


KeyboardInterrupt: Interrupted by user

In [ ]:
I am Promise, Date of birth, B-Tech Computer Science. phone number: 07063083925. I want to learn Excel. promiseibediogwu@gmail.com

In [ ]:
#My name is Promise Ibediogwu, I was born on January 8th 1995, B-Tech Computer Science. This is my phone number: 07063083925. I want to learn Python for Data Analysis, my email is promiseibediogwu@gmail.com

In [ ]:
### storing the user input using NLP entity recognition extraction

In [ ]:
nlp

In [12]:
### implementing conversation track, after the user has entered his or her name

In [65]:
# Function to extract profile data from user input
def extract_profile(user_input):
    doc = nlp(user_input)

    # Apply matcher
    matches = matcher(doc)

    # Initialize extracted data
    name, dob, qualification, phone_number, email, course = None, None, None, None, None, []

    # Process matches
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]

        if label == "NAME":
            name = span.text.replace("My name is", "").strip()
        elif label == "DOB":
            dob = span.text.replace("born on", "").strip()
        elif label == "QUALIFICATION":
            qualification = span.text
        elif label == "PHONE":
            phone_number = span[-1].text

    # Extract email & course via ruler
    for ent in doc.ents:
        if ent.label_ == "EMAIL":
            email = ent.text
        elif ent.label_ == "COURSE":
            course.append(ent.text)

    return name, dob, qualification, phone_number, email, course


In [66]:
extract_profile('My name is Promise Ibediogwu, I was born on January 8th 1995, B-Tech Computer Science. This is my phone number: 07063083925. I want to learn Python for Data Analysis, my email is promiseibediogwu@gmail.com')

('Promise Ibediogwu',
 'January 8th 1995',
 'Data Analysis',
 '07063083925',
 'promiseibediogwu@gmail.com',
 ['Python for Data Analysis'])

## Working code

In [2]:
# -------------------------------------------------------- LIBRARIES -----------------------------------------------
import dateparser
import pickle
import spacy
import numpy as np
import logging
from datetime import datetime
import json
from spacy.pipeline import EntityRuler
import re
from spacy.matcher import Matcher

# Load the saved pipeline
with open("intent_classifier.pkl", "rb") as f:
    loaded_pipeline = pickle.load(f)

# Set up logging to file
logging.basicConfig(
    filename='chat_log.json',
    level=logging.INFO,
    format='%(message)s'
)


In [3]:
# -------------------------------------------------------- DATA ----------------------------------------------------
# Intent mapping
target_response = {
    0: 'greetings',
    1: 'enrollment',
    2: 'pricing',
    3: 'tracking',
    4: 'fallback'
}

# Response templates
fallback_responses = [
    "Sorry, I didn’t quite understand that. Can you rephrase, read my previous prompt and respond accordingly?",
    "Hmm, I'm not sure what you mean. Try asking about enrollment or pricing, read my previous prompt and respond accordingly.",
    "Let’s try that again. Ask about courses, prices, or tracking progress, read my previous prompt and respond accordingly"
]

response = {
    'greetings': [
        "Nice, you are very much welcome to Poshem! What do you want to do?\n"
        "- Do want to enroll/register for a course?\n- Check the price of our courses?\n- Confirm your registered courses?"
    ],
    'enrollment': (
        "Kindly request for the course you want to register in this format:\n"
        "(e.g.) I am Promise, born on Date of birth, B-Tech Computer Science.\n"
        "phone number: 07063083925.\n"
        "email address: promise@x.com\n"
        "I want to learn Excel."
    ),
    'pricing': "Our pricing varies by course. Here is the breakdown...",
    'tracking': "To track your course, kindly let me know your ID.",
    'fallback': fallback_responses
}

In [4]:
#---------------------------------------------------------USER PROFILE EXTRACTOR-----------------------------------------

# Load spaCy model
nlp = spacy.load('en_core_web_lg')

# Initialize EntityRuler
ruler = EntityRuler(nlp, overwrite_ents=True)
 
# Remove default NER to use custom recognition
if "ner" in nlp.pipe_names:
    nlp.remove_pipe("ner")

# Add EntityRuler for course and email
ruler = nlp.add_pipe("entity_ruler", first=True)

# Define custom course list
courses = [
    "excel", "powerbi", "sql", "postgresql", "python for beginners",
    "advance python for data analysis", "python for data analysis",
    "data science", "machine learning",
    "database administrator", "azure devops", "aws devops"
]

course_patterns = [{"label": "COURSE", "pattern": course} for course in courses]
email_patterns = [{"label": "EMAIL", "pattern": [{"LIKE_EMAIL": True}]}]
ruler.add_patterns(course_patterns + email_patterns)

# Add Matcher for name, DOB, qualification, and phone
matcher = Matcher(nlp.vocab)

matcher.add("NAME", [[
    {"LOWER": "i"},
    {"LOWER": "am"},
    {"IS_TITLE": True, "OP": "+"}
]])

matcher.add("DOB", [[
    {"LOWER": "born"},
    {"LOWER": "on"},
    {"IS_TITLE": True, "OP": "+"},
    {"TEXT": {"REGEX": r"^\d{1,2}(st|nd|rd|th)?$"}, "OP": "?"},
    {"IS_DIGIT": True, "OP": "?"}
]])

matcher.add("PHONE", [[
    {"LOWER": "phone"},
    {"LOWER": "number"},
    {"IS_PUNCT": True, "OP": "?"},
    {"TEXT": {"REGEX": r"^\d{11}$"}}
]])

matcher.add("QUALIFICATION", [[
    {"TEXT": {"REGEX": r"^[A-Z][-.A-Za-z]+$"}},
    {"IS_TITLE": True, "OP": "+"}
]])


In [5]:
#-------------------------------------------------------------------BOT FUNCTIONS-----------------------------------------

# Function to simulate intent prediction
def user_input_to_model(user_input):
    vector = nlp(user_input).vector
    predicted_intent = loaded_pipeline.predict([vector])[0]  # Extract single prediction
    return predicted_intent

# Function to generate bot response
def response_function(user_intent):
    intent_label = target_response.get(user_intent, 'fallback')

    if intent_label == 'fallback':
        return np.random.choice(response['fallback'])

    reply = response.get(intent_label)
    return reply[0] if isinstance(reply, list) else reply

# Chat history logging function
def log_interaction(user_input, predicted_intent, bot_response):
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "user_input": user_input,
        "predicted_intent": target_response.get(predicted_intent, 'fallback'),
        "bot_response": bot_response
    }
    logging.info(json.dumps(log_entry))


In [6]:
#-----------------------------------------------------------------USER EXTRACTION FUNCTION-------------------------------------

# Function to extract profile data from user input
def extract_profile(user_input):
    doc = nlp(user_input)

    # Apply matcher
    matches = matcher(doc)

    # Initialize extracted data
    name, dob, qualification, phone_number, email, course = None, None, None, None, None, []

    # Process matches
    for match_id, start, end in matches:
        span = doc[start:end]
        label = nlp.vocab.strings[match_id]

        if label == "NAME":
            name = span.text.replace("I am", "").strip()
        elif label == "DOB":
            dob = span.text.replace("born on", "").strip()
        elif label == "QUALIFICATION":
            qualification = span.text
        elif label == "PHONE":
            phone_number = span[-1].text

    # Extract email & course via ruler
    for ent in doc.ents:
        if ent.label_ == "EMAIL":
            email = ent.text
        elif ent.label_ == "COURSE":
            course.append(ent.text)

    return name, dob, qualification, phone_number, email, course


### the good code

In [7]:
#--------------------------------------------------------POSHBOT------------------------------------------------
def Poshbot():
    print("Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:")

    in_registration = False
    completed_registration = False
    predicted_intent = None

    while True:
        user_input = input("You 🫥: ").strip()

        if user_input.lower() in ["i am done", "exit", "quit"]:
            print("Bot 🤖: Thanks for choosing Poshem Technologies Institution. This is your course ID. You'll get a mail from us shortly.")
            break

        try:
            # Step 1: If user is in registration mode, extract profile
            if in_registration and not completed_registration:
                try:
                    name, dob, qualification, phone_number, email, course = extract_profile(user_input)

                    print("Bot 🤖: Thank you! Here's your profile:")
                    print(f"Name: {name}")
                    print(f"DOB: {dob}")
                    print(f"Qualification: {qualification}")
                    print(f"Phone: {phone_number}")
                    print(f"Email: {email}")
                    print(f"Course: {course}")

                    print("Bot 🤖: Your registration is complete! ✅")

                    print("Bot 🤖: What else do you want to do, else kindly quite, exit or say you are done!!!")

                    completed_registration = True
                    in_registration = False  # Reset flag
                    predicted_intent = None  # Clear intent

                    continue  # Go back to next user input
                except Exception as e:
                    print("Bot 🤖: Sorry, I couldn't extract your enrollment details. Please try again.")
                    continue

            # Step 2: Predict intent if not in registration
            predicted_intent = user_input_to_model(user_input)
            bot_response = response_function(predicted_intent)

            print(f"Bot 🤖: {bot_response}")

            # Step 3: If intent is enrollment, set the flag
            if predicted_intent == 1:  # Or 'enrollment' depending on how your model works
                in_registration = True
                continue

            # Step 4: Handle other general replies
            log_interaction(user_input, predicted_intent, bot_response)

        except Exception as e:
            print("Bot 🤖: Sorry, I couldn't process that. Please try again.")

Poshbot()

Bot 🤖: Hello! Welcome to Poshem Technologies, a fair greeting will be okay for us to attend to you:


You 🫥:  hi


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Do want to enroll/register for a course?
- Check the price of our courses?
- Confirm your registered courses?


You 🫥:  i want to enroll for a course


Bot 🤖: Kindly request for the course you want to register in this format:
(e.g.) I am Promise, born on Date of birth, B-Tech Computer Science.
phone number: 07063083925.
email address: promise@x.com
I want to learn Excel.


You 🫥:  I am Promise Ibediogwu, born on January 8th 1995, B-Tech Computer Science. phone number: 07063083925. email address: promiseibediogwu1@gmail.com, I want to learn python for beginners


Bot 🤖: Thank you! Here's your profile:
Name: Promise Ibediogwu
DOB: January 8th 1995
Qualification: Computer Science
Phone: 07063083925
Email: promiseibediogwu1@gmail.com
Course: ['python for beginners']
Bot 🤖: Your registration is complete! ✅
Bot 🤖: What else do you want to do, else kindly quite, exit or say you are done!!!


KeyboardInterrupt: Interrupted by user

In [ ]:
I am Promise Ibediogwu, born on January 8th 1995, B-Tech Computer Science. phone number: 07063083925. email address: promiseibediogwu1@gmail.com, I want to learn python for beginners

In [ ]:
# Call the function to send the user and order data to the backend
send_user_data_to_db(name, dob, qualification, phone_number, email, course_ordered)


In [2]:
pip install dateparser

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
def Poshbot():
    print("Bot 🤖: Hello! Welcome to Poshem Technologies. A fair greeting will be okay for us to attend to you:")

    in_registration = False
    completed_registration = False
    predicted_intent = None

    while True:
        user_input = input("You 🫥: ").strip()

        if user_input.lower() in ["i am done", "exit", "quit"]:
            print("Bot 🤖: Thanks for choosing Poshem Technologies Institution. This is your course ID. You'll get a mail from us shortly.")
            break

        try:
            # Handle registration flow
            if in_registration and not completed_registration:
                try:
                    name, dob, qualification, phone_number, email, course = extract_profile(user_input)

                    print("Bot 🤖: Thank you! Here's your profile:")
                    print(f"Name: {name}")
                    print(f"DOB: {dob}")
                    print(f"Qualification: {qualification}")
                    print(f"Phone: {phone_number}")
                    print(f"Email: {email}")
                    print(f"Course: {course}")
                    print("Bot 🤖: Your registration is complete! ✅")

                    completed_registration = True
                    in_registration = False

                    # Return to main menu prompt
                    print("Bot 🤖: What else would you like to do?")
                    print("Bot 🤖: You can ask about courses, enroll again, or say 'exit' to quit.")

                    continue
                except Exception as e:
                    print("Bot 🤖: Sorry, I couldn't extract your enrollment details. Please try again.")
                    continue

            # Intent prediction
            predicted_intent = user_input_to_model(user_input)
            bot_response = response_function(predicted_intent)

            print(f"Bot 🤖: {bot_response}")

            if predicted_intent == 1:  # Assuming '1' is the enrollment intent
                in_registration = True
                completed_registration = False  # Reset in case of re-enrollment
                continue

            log_interaction(user_input, predicted_intent, bot_response)

        except Exception as e:
            print("Bot 🤖: Sorry, I couldn't process that. Please try again.")

Poshbot()

Bot 🤖: Hello! Welcome to Poshem Technologies. A fair greeting will be okay for us to attend to you:


You 🫥:  hi


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Do want to enroll/register for a course?
- Check the price of our courses?
- Confirm your registered courses?


You 🫥:  enrollmen


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Do want to enroll/register for a course?
- Check the price of our courses?
- Confirm your registered courses?


You 🫥:  I am Promise Ibediogwu, born on January 8th 1995, B-Tech Computer Science. phone number: 07063083925. email address: promiseibediogwu1@gmail.com, I want to learn python for beginners


Bot 🤖: Nice, you are very much welcome to Poshem! What do you want to do?
- Do want to enroll/register for a course?
- Check the price of our courses?
- Confirm your registered courses?


KeyboardInterrupt: Interrupted by user